# NBA 2017 [Hackathon](https://hackathon.nba.com/)

You should use the data provided in this [LINK](https://ak-static-int.nba.com/wp-content/uploads/sites/2/2017/06/Business-Track-Application-Datasets.xlsx). Suppose you are given a sample data set of 1,000 individuals that includes their last twelve months’ worth of total revenue spend, games watched, income, fan satisfaction (self-reported, 1 to 7, 7 is best), logged complaints and number of years in our database at present (in the Training Data tab). You intend to predict spend metrics for another data set of 100 people whose revenue spend is unknown (in the Prediction tab). Construct a model for 1) A projected twelve-month total revenue sepnd number 2) The probability that total revenue spend is at least $250. Run each of the accounts in the prediction tab through both models and fill in your results in 1) the TotRevSpend column on the Prediction tab and in 2) the LikelihoodOver250 column on the Prediction tab.

In [3]:
#Metadata
%load_ext watermark
%watermark -a "Vutsal Singhal"
%watermark -u -n -t -z
%watermark -p numpy,sklearn,matplotlib,pandas
%watermark -m

Vutsal Singhal
last updated: Wed Aug 02 2017 14:42:22 IST
numpy 1.13.0
sklearn 0.18.2
matplotlib 2.0.2
pandas 0.20.3
compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.10.0-28-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("Business-Track-Application-Datasets.xlsx")
df.head()

,Unnamed: 0,HomeTeam,Round1_Gate,Round2_Gate,Round3_Gate,Round4_Gate
0,NaN,East1,2756446,3528250.88,5.574636e+06,8.417701e+06
1,NaN,East2,722126,888214.98,1.163562e+06,1.838427e+06
2,NaN,East3,642409,732346.26,1.230342e+06,2.079278e+06
3,NaN,East4,3232910,4558403.10,7.384613e+06,1.270153e+07
4,NaN,East5,1780942,2546747.06,3.845588e+06,6.152941e+06
